## secondary model which is trained on masks of first model

For this Notebook it is necessary to add the output data from fynn's Notebook.These are mask outputs of every timestep.

You can access the data of fynns Notebook here: https://www.kaggle.com/fynnjunge/contrails-apply-model-on-all-timesteps

The filepath should look like this.
/kaggle/input/contrails-apply-model-on-all-timesteps

In addition it is necessary to add the data from the competetion to gain access to the correct/official labels/masks.

In [3]:
import sys
sys.path.append("/kaggle/input/smp-github/segmentation_models.pytorch-master")
sys.path.append("/kaggle/input/pretrained-models-pytorch")
sys.path.append("/kaggle/input/efficientnet-pytorch")

In [2]:
import torch
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
from torchmetrics.functional import dice
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
import os
import numpy as np
import torch
import torchvision.transforms as T

from torch.utils.data import Dataset, DataLoader

TRAIN_PATH = "/kaggle/input/contrails-apply-model-on-all-timesteps/train/"
VALIDATION_PATH = "/kaggle/input/contrails-apply-model-on-all-timesteps/validation/"
PATH = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/" #For Labels

class MasksDataset(Dataset):
    def __init__(self, split="train", mode="single"):
        self.split = split
        self.mode = mode
        self.path = TRAIN_PATH if split == "train" else VALIDATION_PATH
        self.maskpath = f'{PATH}{split}/'
        self.examples = os.listdir(self.path)
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
    def read_record(self, filename):
        record_data = np.load(filename)["arr_0"]
        return record_data

    
    def __getitem__(self, index):
        filename = os.path.join(self.path, self.examples[index])
        data = self.read_record(filename)
        
        img = data
        
        if self.mode == "single":
            img = torch.tensor(np.expand_dims(img, axis=0)).to(torch.float32)
        else:
            img = torch.tensor(img).to(torch.float32)
        
        #img = self.normalize_image(img)
        
        if self.split in ["train", "validation"]:
            label = np.load(os.path.join(self.maskpath, f"{self.examples[index][:-4]}/human_pixel_masks.npy")).squeeze()
            label = torch.Tensor(label).to(torch.int64)
            return img.float(), label
        
        return img.float()
    
    def __len__(self):
        return len(self.examples)

In [ ]:
# config
config = {
    "data_path": "/kaggle/input/contrails-images-ash-color", # only used for the labels
    "model": {
        "encoder_name": "None", # not used!
        "loss_smooth": 1.0,
        "optimizer_params": {"lr": 0.003, "weight_decay": 0.01},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 500, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.31622776601,
                    "mode": "min",
                    "patience": 4,
                    "verbose": True,
                },
            },
        },
        "seg_model": "Linknet",
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 50,
    "seed": 42,
    "train_bs": 32,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 30,
        "min_epochs": 30,
    },
    "valid_bs": 64,
    "workers": 2,
}

In [ ]:
mask_train_dataset = MasksDataset(split="train", mode="multi")
mask_validation_dataset = MasksDataset(split="validation", mode="multi")

In [ ]:
data_loader_train = DataLoader(
    mask_train_dataset, batch_size=config["train_bs"], shuffle=True, num_workers=config["workers"]
)
data_loader_validation = DataLoader(
    mask_validation_dataset, batch_size=config["valid_bs"], shuffle=False, num_workers=config["workers"]
)

pl.seed_everything(config["seed"])

In [ ]:
mask_train_dataset[0][0].shape

In [ ]:
mask_train_dataset[0][1].shape

In [ ]:
from pprint import pprint
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar

In [ ]:
# Lightning module

seg_models = {
    "Unet": smp.Unet,
    "Unet++": smp.UnetPlusPlus,
    "MAnet": smp.MAnet,
    "Linknet": smp.Linknet,
    "FPN": smp.FPN,
    "PSPNet": smp.PSPNet,
    "PAN": smp.PAN,
    "DeepLabV3": smp.DeepLabV3,
    "DeepLabV3+": smp.DeepLabV3Plus,
}


class LightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = model = seg_models[config["seg_model"]](
            #encoder_name=config["encoder_name"], # no encoder name
            encoder_weights=None,  # random wncoder weights
            in_channels=8,
            classes=1,
            activation=None,
        )
        self.loss_module = smp.losses.DiceLoss(mode="binary", smooth=config["loss_smooth"])
        self.val_step_outputs = []
        self.val_step_labels = []

    def forward(self, batch):
        imgs = batch
        preds = self.model(imgs)
        return preds

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config["optimizer_params"])

        if self.config["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=16)

        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.val_step_outputs.append(preds)
        self.val_step_labels.append(labels)

    def on_validation_epoch_end(self):
        all_preds = torch.cat(self.val_step_outputs)
        all_labels = torch.cat(self.val_step_labels)
        self.val_step_outputs.clear()
        self.val_step_labels.clear()
        val_dice = dice(all_preds, all_labels.long())
        self.log("val_dice", val_dice, on_step=False, on_epoch=True, prog_bar=True)

In [ ]:
logger = pl.loggers.CSVLogger("csv_log")

In [ ]:
filename = f"model"

checkpoint_callback = ModelCheckpoint(
    monitor="val_dice",
    dirpath=config["output_dir"],
    mode="max",
    filename=filename,
    save_top_k=1,
    verbose=1,
)

# Training

In [ ]:
progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)

trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback], logger=logger, **config["trainer"]
)

In [ ]:
model = LightningModule(config["model"])

In [ ]:
trainer.fit(model, data_loader_train, data_loader_validation)

# Predictions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
model.zero_grad()

In [ ]:
examples_to_check = [100, 80, 64, 42, 3]

nrows, ncols = (5,2)

fig, axs = plt.subplots(nrows, ncols, figsize = (16,16))

for i, img in enumerate(examples_to_check):
    label = mask_train_dataset[img][1]
    preds = model(torch.unsqueeze(mask_train_dataset[img][0].to(device), 0))
    
    
    axs[i, 0].imshow(label,cmap='gray')
    axs[i, 1].imshow(preds.to("cpu").detach().squeeze(),cmap='gray')
        
plt.show()

del preds

In [ ]:
# more examples
examples_to_check = [100, 81, 65, 43, 4]

nrows, ncols = (5,2)

fig, axs = plt.subplots(nrows, ncols, figsize = (16,16))

for i, img in enumerate(examples_to_check):
    label = mask_train_dataset[img][1]
    preds = model(torch.unsqueeze(mask_train_dataset[img][0].to(device), 0))    
    
    axs[i, 0].imshow(label)
    axs[i, 1].imshow(preds.to("cpu").detach().squeeze())
        
plt.show()

del preds

# Further explorations of output

In [ ]:
mask_train_dataset[100][1].sum()

In [ ]:
mask_train_dataset[100][1]

In [ ]:
torch.unique(mask_train_dataset[100][1])

In [ ]:
torch.unique(mask_train_dataset[100][0])

In [ ]:
my_tensor80 = mask_train_dataset[80][0]
my_tensor100 = mask_train_dataset[100][0]

In [ ]:
# Convert the tensor to a NumPy array
my_np_array = my_tensor80.numpy()

# Compute the histogram using numpy.histogram()
# You can specify the number of bins using the `bins` parameter.
hist_values, hist_bins = np.histogram(my_np_array, bins=np.arange(my_np_array.min(), my_np_array.max() + 2))

# Plot the histogram
plt.bar(hist_bins[:-1], hist_values, align='center', width=0.8)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values within the Tensor')
plt.show()


In [ ]:

# Convert the tensor to a NumPy array
my_np_array = my_tensor100.numpy()

# Compute the histogram using numpy.histogram()
# You can specify the number of bins using the `bins` parameter.
hist_values, hist_bins = np.histogram(my_np_array, bins=np.arange(my_np_array.min(), my_np_array.max() + 2))

# Plot the histogram
plt.bar(hist_bins[:-1], hist_values, align='center', width=0.8)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values within the Tensor')
plt.show()


In [ ]:

# Convert the tensor to a NumPy array
my_np_array = mask_train_dataset[64][0].numpy()

# Compute the histogram using numpy.histogram()
# You can specify the number of bins using the `bins` parameter.
hist_values, hist_bins = np.histogram(my_np_array, bins=np.arange(my_np_array.min(), my_np_array.max() + 2))

# Plot the histogram
plt.bar(hist_bins[:-1], hist_values, align='center', width=0.8)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values within the Tensor')
plt.show()


## How do we convert the data into binary data?